In [28]:
import torch
import numpy as np
import torch.functional as F
import torch.nn as nn
from nltk import ngrams
from IPython.display import display
import pandas as pd
from tqdm import tqdm

from nltk.tokenize import sent_tokenize, wordpunct_tokenize, word_tokenize
from gensim.models import KeyedVectors
from utils import get_distinct_words, read_corpus
from itertools import chain

In [3]:
print(torch.device('cuda:1'))
print(torch.cuda.device(0))

cuda:1


In [31]:
min_count = 2
ru_corpus_cp = read_corpus("ru_copy")
index_to_key, word_counter = get_distinct_words(ru_corpus_cp, min_count=min_count)
index_to_key = ["UNK", "PAD"] + index_to_key
key_to_index = {word: i for i, word in enumerate(index_to_key)}

In [8]:
len(ru_corpus_cp), ru_corpus_cp[:2]

(8,
 [['кстати',
   'как',
   'неожиданно',
   'кпрф',
   'становиться',
   'не',
   'все',
   'равный',
   'на',
   'судьба',
   'фермер',
   'именно',
   'накануне',
   'выборы'],
  ['можно',
   'и',
   'по',
   'другому',
   'сказать',
   'убогий',
   'клоунада',
   'кпрф',
   'это',
   'попытка',
   'отвечать',
   'на',
   'запрос',
   'молодой',
   'поколение',
   'левый',
   'не',
   'питать',
   'иллюзия',
   'по',
   'повод',
   'коммунистический',
   'номенклатура',
   'советский',
   'образец',
   'но',
   'в',
   'сила',
   'свой',
   'положение',
   'под',
   'давление',
   'вызов',
   'время',
   'они',
   'вынуждать',
   'быть',
   'меняться']])

In [10]:
def as_matrix(sequences, key_to_index, UNK="UNK", PAD="PAD", max_len=None):
    """ Convert a list of tokens into a matrix with padding """
    if isinstance(sequences[0], str):
        sequences = [x.split() for x in sequences]

    max_sequence_len = max([len(x) for x in sequences])
    if max_len is not None and max_sequence_len > max_len :
        max_sequence_len = max_len

    matrix = np.full((len(sequences), max_sequence_len), np.int32(key_to_index[PAD]))
    for i, seq in enumerate(sequences):
        row_ix = [key_to_index.get(word, key_to_index[UNK]) for word in seq[:max_sequence_len]]
        matrix[i, :len(row_ix)] = row_ix

    return matrix

In [16]:
display(len(ru_corpus_cp))
# display(as_matrix(ru_corpus_cp, key_to_index, max_len=10))
len(list(chain.from_iterable(ru_corpus_cp)))

8

309

In [22]:
class BaseEmbeddings(KeyedVectors):
    def __init__(self, corpus, distinct_words=None, word_counter=None, vector_size=100, min_count=10):
        super().__init__(vector_size=vector_size)
        
        self.index_to_key = distinct_words
        self.word_counter = word_counter
        if distinct_words is None or word_counter is None:
            self.index_to_key, self.word_counter = get_distinct_words(corpus, min_count=min_count)
    
        self.key_to_index = {word: i for i, word in enumerate(self.index_to_key)}

In [44]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def pad_text(text: list, window_size: int, pad: str):
    appendix = [pad] * window_size

    return appendix + text + appendix

In [100]:
np.random.choice(50, 20)


array([34, 38, 19, 32, 24, 13, 40, 29, 14, 24, 46, 20, 30, 49, 26, 44, 33,
       46, 46,  1])

In [129]:
np.arange(5)[np.newaxis, ...].shape, np.arange(10)[np.newaxis, ...].shape

((1, 5), (1, 10))

In [130]:
np.arange(5)[..., np.newaxis] @ np.arange(10)[np.newaxis, ...]

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18],
       [ 0,  3,  6,  9, 12, 15, 18, 21, 24, 27],
       [ 0,  4,  8, 12, 16, 20, 24, 28, 32, 36]])

In [117]:
class Word2Vec(BaseEmbeddings):
    def __init__(self, corpus, distinct_words=None, vector_size=100, window_size=5,
                 min_count=10, batch_size=None, n_negative=5, n_epoches=5):
        super().__init__(corpus, vector_size=vector_size, distinct_words=distinct_words, min_count=min_count)

        self.V = torch.randn((len(self.index_to_key), vector_size))  # vocab_size, vector_size    #, device=torch.cuda.device(0))
        self.U = torch.randn((len(self.index_to_key), vector_size))  # vocab_size, vector_size    #, device=torch.cuda.device(0))

        # UNK, PAD = "UNK", "PAD"
        # self.index_to_key = [UNK, PAD] + self.index_to_key

        self.corpus = corpus
        self.squeeze = list(chain.from_iterable(corpus))
        self.window_size = window_size
        self.batch_size = batch_size
        if batch_size is None:
            self.batch_size = np.max([len(text) for text in corpus])
        self.n_negative = n_negative
        self.alpha = 0.001
        
        self.train(n_epoches)
        self.vectors = self.V
    

    def train(self, n_epoches=5):
        """
        trains self.center_W and self.context_W matrices
        """
        print(len(self.key_to_index), len(self.index_to_key))
        for epoch in tqdm(range(n_epoches)):
            for batch in chunks(self.squeeze, self.batch_size):
                text = pad_text(batch, window_size=self.window_size, pad="UNK")
                for ind in range(len(text) - 2 * self.window_size): 
                    start = ind
                    end = start + 2 * self.window_size
                    w_j = start + self.window_size
                    if text[w_j] in self.key_to_index.keys():
                        for c in range(start, end + 1):
                            if c != w_j and text[c] in self.key_to_index.keys():
                                # print(self.key_to_index[text[w_j]], self.key_to_index[text[c]])
                                # print(text[w_j], text[c])
                                k_neg = np.random.choice(len(self.index_to_key), self.n_negative)  # take UNK PAD
                                # print(torch.sigmoid(
                                print(k_neg)
                                # self.V[self.key_to_index[text[w_j]]
                                U_k = self.U[k_neg]
                                V_j = self.V[self.key_to_index[text[w_j]]]
                                print((U_k @ V_j) * V_j)  # n_negative, vector_size x vector_size
                                # self.np.random.choice(len(self.squeeze), self.n_negative)
                        break


                    # self.V -= self.alpha * dJ_dV
                    # self.U -= self.alpha * dJ_dU

In [118]:
w2v = Word2Vec(ru_corpus_cp, min_count=2)

50 50


  0%|          | 0/5 [00:00<?, ?it/s]

[ 2 32  1 15 31]


RuntimeError: The size of tensor a (5) must match the size of tensor b (100) at non-singleton dimension 0

In [40]:
w2v.squeeze

['кстати',
 'как',
 'неожиданно',
 'кпрф',
 'становиться',
 'не',
 'все',
 'равный',
 'на',
 'судьба',
 'фермер',
 'именно',
 'накануне',
 'выборы',
 'можно',
 'и',
 'по',
 'другому',
 'сказать',
 'убогий',
 'клоунада',
 'кпрф',
 'это',
 'попытка',
 'отвечать',
 'на',
 'запрос',
 'молодой',
 'поколение',
 'левый',
 'не',
 'питать',
 'иллюзия',
 'по',
 'повод',
 'коммунистический',
 'номенклатура',
 'советский',
 'образец',
 'но',
 'в',
 'сила',
 'свой',
 'положение',
 'под',
 'давление',
 'вызов',
 'время',
 'они',
 'вынуждать',
 'быть',
 'меняться',
 'вот',
 'он',
 'тонкий',
 'незаметный',
 'ход',
 'против',
 'россия',
 'зюганов',
 'какой',
 'же',
 'жук',
 'давать',
 'пища',
 'для',
 'будущий',
 'раздор',
 'под',
 'благовидный',
 'предлог',
 'прикрываться',
 'бог',
 'нет',
 'слово',
 'просто',
 'в',
 'этот',
 'паблик',
 'рано',
 'подобный',
 'пост',
 'не',
 'замечаться',
 'видимо',
 'с',
 'непривычка',
 'а',
 'к',
 'выходка',
 'кпрф',
 'я',
 'уже',
 'привыкать',
 'не',
 'реагировать',